In [ ]:
!pip install --upgrade pip
!pip install lxml
!pip install spacy-langdetect

In [ ]:
import requests
from lxml import etree
from io import StringIO
import pandas as pd
import spacy
from spacy.language import Language
from spacy_langdetect import LanguageDetector
import re
import pprint
from spacy import displacy
from collections import Counter

In [3]:
@Language.factory("language_detector")
def get_lang_detector(nlp, name):
   return LanguageDetector()

In [ ]:
# !python -m spacy download xx_ent_wiki_sm
!python -m spacy download en_core_web_lg

In [ ]:
# nlp_ner = spacy.load("xx_ent_wiki_sm")
nlp = spacy.load("en_core_web_lg")
nlp.add_pipe("language_detector", last=True)

In [ ]:
index_url = 'https://alpha-vlo.clarin.eu/data/clarin/results/cmdi/COllections_de_COrpus_Oraux_Numeriques_CoCoON_ex_CRDO_/'
response = requests.get(index_url)

if response.status_code == 200:
    html = response.content.decode('utf-8')
    parser = etree.HTMLParser()
    tree = etree.fromstring(html, parser)

    # Find all the <a> tags with class="result-item"
    xml_links = tree.xpath('//a[@href and substring(@href, string-length(@href) - 3)=".xml"]/@href')
    
    with open('descriptions.txt', 'w', encoding='utf-8') as f:
      for xml_link in xml_links:
        xml_url = index_url + xml_link
        xml_response = requests.get(xml_url)
        if xml_response.status_code == 200:
          xml_content = xml_response.content
          xml_parser = etree.XMLParser()
          xml_tree = etree.fromstring(xml_content, xml_parser)
          try:
            description = xml_tree.xpath('//cmd:CMD/cmd:Components/cmdp:OLAC-DcmiTerms/cmdp:description//text()', 
                                          namespaces={"cmd": "http://www.clarin.eu/cmd/1",
                                                      "cmdp": "http://www.clarin.eu/cmd/1/profiles/clarin.eu:cr1:p_1288172614026",
                                                      "xsi": "http://www.w3.org/2001/XMLSchema-instance"})[0]
            description = str(description)
            description = re.sub(r"\n {2,}", '', description)
            description = re.sub("\n\t+", ' ', description)
            description = re.sub("\0+", '', description)
            description = re.sub(r"\n", ' ', description)
            doc = nlp(description)
            language = doc._.language["language"]
            confidence_score = doc._.language["score"]
            f.write(f"{language}\t{confidence_score}\t{doc}\n")
          except:
              IndexError
              continue
 
else:
    print(f"Error: {response.status_code}")

In [ ]:
with open(r'/content/descriptions.txt', 'r') as f:
    lines = f.readlines()

data = [line.strip().split('\t') for line in lines]

df = pd.DataFrame(data, columns=['Language', 'Confidence Score', 'Description'])

df.to_csv('descriptions_with_lang.csv', encoding='utf-8', index=False)

In [ ]:
with open("descriptions.txt", "r") as file:
  contents = file.read()

doc = nlp(contents)
# for ent in doc.ents:
displacy.render(nlp(str(doc)), jupyter=True, style='ent')

In [ ]:
pprint.pprint([(X.text, X.label_) for X in doc.ents])